In [10]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

In [11]:
train_path='/content/data/train'
pred_path='/content/data/val'

In [13]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [14]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()

        #Output size after convolution filter
        #((w-f+2P)/s) +1

        #Input shape= (256,3,150,150)

        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)

        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)


        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)



        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)


        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)



        #Feed forwad function

    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)

        output=self.pool(output)

        output=self.conv2(output)
        output=self.relu2(output)

        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)


            #Above output will be in matrix form, with shape (256,32,75,75)

        output=output.view(-1,32*75*75)


        output=self.fc(output)

        return output

In [17]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=4)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=4, bias=True)
)

In [19]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [20]:
#prediction function
def prediction(img_path,transformer):

    image=Image.open(img_path)

    image_tensor=transformer(image).float()


    image_tensor=image_tensor.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor.cuda()

    input=Variable(image_tensor)


    output=model(input)

    index=output.data.numpy().argmax()

    pred=classes[index]

    return pred



In [35]:
images_path=glob.glob(pred_path+'/**/*.png')
print(images_path)

['/content/Data Main/val/Rocky/Rocky (15)_0_16.png', '/content/Data Main/val/Rocky/Rocky (14)_12_12.png', '/content/Data Main/val/Rocky/Rocky (1)_0_10.png', '/content/Data Main/val/Rocky/Rocky (9)_2_19.png', '/content/Data Main/val/Rocky/Rocky (20)_7_26.png', '/content/Data Main/val/Rocky/Rocky (13)_13_12.png', '/content/Data Main/val/Rocky/Rocky (12)_0_4.png', '/content/Data Main/val/Rocky/Rocky (16)_6_24.png', '/content/Data Main/val/Rocky/Rocky (8)_12_14.png', '/content/Data Main/val/Rocky/Rocky (8)_6_10.png', '/content/Data Main/val/Rocky/Rocky (11)_15_6.png', '/content/Data Main/val/Rocky/Rocky (20)_1_0.png', '/content/Data Main/val/Rocky/Rocky (13)_2_23.png', '/content/Data Main/val/Rocky/Rocky (5)_12_12.png', '/content/Data Main/val/Rocky/Rocky (16)_15_18.png', '/content/Data Main/val/Rocky/Rocky (20)_10_29.png', '/content/Data Main/val/Rocky/Rocky (14)_1_9.png', '/content/Data Main/val/Rocky/Rocky (8)_0_11.png', '/content/Data Main/val/Rocky/Rocky (11)_2_1.png', '/content/Data 

In [33]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [34]:
pred_dict

{'Rocky (15)_0_16.png': 'Rocky',
 'Rocky (14)_12_12.png': 'Rocky',
 'Rocky (1)_0_10.png': 'Marshy',
 'Rocky (9)_2_19.png': 'Rocky',
 'Rocky (20)_7_26.png': 'Rocky',
 'Rocky (13)_13_12.png': 'Rocky',
 'Rocky (12)_0_4.png': 'Rocky',
 'Rocky (16)_6_24.png': 'Grassy',
 'Rocky (8)_12_14.png': 'Rocky',
 'Rocky (8)_6_10.png': 'Grassy',
 'Rocky (11)_15_6.png': 'Rocky',
 'Rocky (20)_1_0.png': 'Rocky',
 'Rocky (13)_2_23.png': 'Rocky',
 'Rocky (5)_12_12.png': 'Rocky',
 'Rocky (16)_15_18.png': 'Rocky',
 'Rocky (20)_10_29.png': 'Rocky',
 'Rocky (14)_1_9.png': 'Marshy',
 'Rocky (8)_0_11.png': 'Rocky',
 'Rocky (11)_2_1.png': 'Rocky',
 'Rocky (2)_5_10.png': 'Rocky',
 'Rocky (9)_8_10.png': 'Rocky',
 'Rocky (5)_0_29.png': 'Rocky',
 'Rocky (5)_3_12.png': 'Rocky',
 'Rocky (6)_10_9.png': 'Rocky',
 'Rocky (7)_9_22.png': 'Rocky',
 'Rocky (11)_6_3.png': 'Rocky',
 'Rocky (20)_5_6.png': 'Rocky',
 'Rocky (5)_4_3.png': 'Marshy',
 'Rocky (5)_5_8.png': 'Rocky',
 'Rocky (17)_10_23.png': 'Marshy',
 'Rocky (11)_0_14.p